In [57]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136599
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-136599


In [80]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "my-cluster"

try:
    cluster=ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('found cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, idle_seconds_before_scaledown=3000)
    cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)


found cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [88]:
from azureml.widgets import RunDetails
from azureml.core import Environment
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--first-layer-neurons': choice(10,50,200,500),
        '--second-layer-neurons': choice(10,50,200,300,500),
        '--batch-size': choice(25,50,100),
        '--learning-rate':uniform(-10,-1),
             
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3 , slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# script_params={'--kernel':'linear',
#                '--penalty': 1.0,
# }
# est = SKLearn(source_directory='.',
#               entry_script='train(1).py',
#               script_params=script_params,
#               compute_target=cluster,
#              )
# sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name= 'AzureML-Tutorial')
target = ws.compute_targets['my-cluster']
est = ScriptRunConfig(source_directory='.',
              script='train(1).py',
              arguments=['--kernel', 'linear', '--penalty', 1.0],
              compute_target=target,
              environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdriveconfig = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    max_concurrent_runs=4,
                                    primary_metric_name='Accuracy',
                                    max_total_runs=20,
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [89]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdriverun= Experiment(ws,'udacity-project').submit(config=hyperdriveconfig)
RunDetails(hyperdriverun).show()

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

In [87]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###